# Building cancer-detection system with RadIO

In [2]:
import numpy as np
import pandas as pd

## 1. What is Lung Cancer?

* CT-scans are 3d-arrays of data

* Luna 

## 2. Dealing with imperfect annotation

## 3. Building detection system-workflow

### `Dataset` and `Pipeline`

CT-scans are voluminous. It is not possible to store all scans from the dataset in memory at once. In order to work with the dataset of scans, we create `Dataset`: an *indexing strructure*:

In [5]:
from radio.dataset import FilesIndex, Dataset, Pipeline
from radio import CTImagesMaskedBatch

LUNA_MASK = '/data/MRT/luna/s*/*.mhd'                     # set glob-mask for scans from Luna-dataset here
luna_index = FilesIndex(path=LUNA_MASK, no_ext=True)      # preparing indexing structure
luna_dataset = Dataset(index=luna_index, batch_class=CTImagesMaskedBatch)

Data-processing workflows are called `pipelines`, which represent sequences(plans) of actions on data

In [ ]:
some_pipeline = (Pipeline()
                 .load()
                 .preprocess_somehow()
                 ....)

To perform some real computation we need to pass a batch through a `pipeline`:

In [ ]:
batch = (dataset >> some_pipeline).next_batch(3)

### 3.1 Preprocessing

In [1]:
ANNO_PATH = '/data/MRT/luna/annotation/annotation.csv'

In [ ]:
noduels_df = pd.read_csv(ANNO_PATH)

In [ ]:
preprocessing = (Pipeline()
                 .load(fmt='blosc')
                 .fetch_nodules_info(ANNO_PATH)
                 .create_mask()
                 .normalize_hu())

In [ ]:
batch = (lunaset >> preprocessing).next_batch(3)

*OOOOH LOOKS LIKE THE SCALES ARE DEAD DIFFERENT? MAYBE FIXING IT? GOOD WE HAVE `UNIFY_SPACING`*

In [ ]:
SHAPE = (256, 256, 256)
SPACING = (1.7, 1., 1.)
preprocessing += Pipeline().unify_spacing()
batch = (lunaset >> preprocessing).next_batch(3)

*OOU IT"S ALL OK NOW THE SCALES ARE THE SAME*

### 3.2 Augmentation

### 3.3 Model Training

In [ ]:
import tensorflow as tf
from radio.dataset import B, V, F
from radio.dataset.models.tf import TFModel

model_path = 'unet/e_6_3c_unet/models/logloss/'
config = dict(load=dict(path=model_path),
              session=dict(config=tf.ConfigProto(allow_soft_placement=True)))


SHAPE = (128, 256, 256)
SPACING = (1.7, 1, 1)
XIP_PARAMS = dict(mode='max', depth=6, stride=2, channels=3)

ppl_predict_scan = (Pipeline()
                    .init_model('static', TFModel, 'xipnet', config)
                    .load(fmt='blosc', components=['images', 'spacing', 'origin', 'masks'])
                    .fetch_nodules_from_mask()
                    .init_variables(['predictions', 'nodules_true', 'nodules_predicted'])
                    .fetch_nodules_from_mask()
                    .update_variable('nodules_true', B('nodules'))
                    .predict_model('xipnet', save_to=V('predictions'),
                                   feed_dict=dict(images=F(CTIMB.xip, component='images', **XIP_PARAMS)))
                    .call(CTIMB.unxip, xip=V('predictions'), squeeze=True, **XIP_PARAMS,
                          component='masks')
                    .fetch_nodules_from_mask()
                    .update_variable('nodules_predicted', B('nodules')) << lunaset)


## THE END

### -----------------------------------------------------------------

In [13]:
pipe = (Pipeline()
        .load(fmt='blosc', components=['images', 'masks', 'spacing', 'origin']))

In [14]:
bloscix = FilesIndex(path="/home/alexander/Work/Temp/unrarred/needed/other/_demo_data/scans_sample/*",
                     dirs=True)
bloscset = Dataset(bloscix, CTImagesMaskedBatch)

In [15]:
bch = (bloscset >> pipe).next_batch(2)

In [20]:
import pickle as pkl

In [21]:
with open('/home/alexander/Загрузки/non_merged.pkl', 'rb') as file:
    non_merged = pkl.load(file)

In [25]:
non_merged













,AccessionNumber,DoctorID,NoduleID,NoduleType,coordX,coordY,coordZ,diameter_mm
0,AGFA000000004788,008,6c132233457ee23fc8b1,ground_glass,284.700,133.250,214.0,7.0
1,AGFA000000004788,008,aee956cbacbed33ff48a,solid,278.200,139.750,189.0,3.0
2,AGFA000000004788,008,72ecdac51c5ee93f6021,solid,272.350,143.000,168.0,9.0
3,AGFA000000004788,008,435d78f78658e73ff4c1,solid,256.750,141.050,147.0,11.0
4,AGFA000000004788,008,ee36d31c700dd23f63fb,solid,267.150,179.400,141.0,8.0
5,AGFA000000004788,008,c8d86f5e2a73cb3fbc12,solid,267.800,170.300,136.0,9.0
6,AGFA000000004788,008,40e3a3d54bbbc83f12d1,solid,273.650,163.800,126.0,12.0
7,AGFA000000004788,008,2473be89318bc83f60f9,solid,268.450,148.200,126.0,18.0
8,AGFA000000004788,008,fb102a3daa27ee3fe0de,solid,232.700,137.800,119.0,13.0
9,AGFA000000004788,008,8529037d4895e83f9c12,solid,228.800,150.800,119.0,10.0
